In [1]:
import os
import string
import torch
from torch import nn
from torch.nn import functional as F
if not "CHDIR_FLAG" in dir():
    os.chdir("../")
    CHDIR_FLAG = True
    
import numpy as np
import pandas as pd
from safetensors import safe_open
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM


from src.tools.plot import plot_tensor_mean_and_variance, plot_tensor_histogram, plot_tensor_heatmap
from src.tools.torch import register_forward_hook_decorator, register_backward_hook_decorator

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

# Preliminary

1. 在之前的研究中，我们发现靠后的层具有更重要的作用？是否有可能丢弃一些不重要的层？
2. 同理在